In [ ]:
:build_env PKG_CONFIG_PATH=/home/jmagin/.local/opt/mambaforge/envs/cdshealpix/lib/pkgconfig
:dep cdshealpix
:dep rayon
:dep geographiclib-rs
:dep geodesy
:dep arrow
:dep parquet

In [ ]:
use cdshealpix::compass_point::Cardinal;
use cdshealpix::nested;
use geodesy::{math::series::FourierCoefficients, Ellipsoid};
use geographiclib_rs::{Geodesic, PolygonArea, Winding};
use rayon::prelude::*;

In [ ]:
fn cell_area(boundary: Vec<(f64, f64)>, g: &Geodesic) -> f64 {
    let mut pa = PolygonArea::new(g, Winding::CounterClockwise);

    for (lon, lat) in boundary {
        pa.add_point(lat, lon);
    }

    let (_, area, _) = pa.compute(false);

    area
}

In [ ]:
fn compute_boundary(cell_id: u64, layer: &nested::Layer, step: u32) -> Vec<(f64, f64)> {
    nested::path_along_cell_edge(layer.depth(), cell_id, &Cardinal::S, false, step)
        .into_iter()
        .map(|p| (p.0.to_degrees(), p.1.to_degrees()))
        .collect::<Vec<(f64, f64)>>()
}

In [ ]:
fn authalic_to_geographic(
    boundary: Vec<(f64, f64)>,
    ellipsoid: &Ellipsoid,
    coefficients: &FourierCoefficients,
) -> Vec<(f64, f64)> {
    boundary
        .into_iter()
        .map(|p| {
            (
                p.0,
                ellipsoid
                    .latitude_authalic_to_geographic(p.1.to_radians(), coefficients)
                    .to_degrees(),
            )
        })
        .collect::<Vec<(f64, f64)>>()
}

In [ ]:
use arrow::error::ArrowError;
use arrow::array::{Float64Array, UInt64Array};
use arrow::array::RecordBatch;
use std::sync::Arc;
use arrow::array::Array;

let wgs84 = Geodesic::wgs84();
let sphere = Geodesic::new(6371000f64, 0f64);

let ellipsoid = Ellipsoid::named("WGS84").unwrap();
let coefficients = ellipsoid.coefficients_for_authalic_latitude_computations();

fn compute_area_stats(
    layer: &nested::Layer,
    step: u32,
    sphere: &Geodesic,
    wgs84: &Geodesic,
    ellipsoid: &Ellipsoid,
    coefficients: &FourierCoefficients,
) -> Result<RecordBatch, ArrowError> {
    let cell_ids = (0..layer.n_hash()).collect::<Vec<u64>>();
    let areas_sphere = cell_ids.par_iter().map(|&hash| compute_boundary(hash, layer, step)).map(|boundary| cell_area(boundary, sphere)).collect::<Vec<f64>>();
    let areas_wgs84 = cell_ids.par_iter().map(|&hash| compute_boundary(hash, layer, step)).map(|boundary| cell_area(boundary, wgs84)).collect::<Vec<f64>>();

    let areas_wgs84_authalic = cell_ids.par_iter()
        .map(|&hash| compute_boundary(hash, layer, step))
        .map(|boundary| authalic_to_geographic(boundary, ellipsoid, coefficients))
        .map(|boundary| cell_area(boundary, wgs84))
        .collect::<Vec<f64>>();

    let cell_ids_ = Arc::new(UInt64Array::from_iter(cell_ids.iter().cloned())) as _;
    let areas_sphere_ = Arc::new(Float64Array::from_iter(areas_sphere.iter().cloned())) as _;
    let areas_wgs84_ = Arc::new(Float64Array::from_iter(areas_wgs84_authalic.iter().cloned())) as _;
    let areas_lazy_wgs84_ = Arc::new(Float64Array::from_iter(areas_wgs84.iter().cloned())) as _;

    RecordBatch::try_from_iter([
        ("cell_id", cell_ids_),
        ("areas_sphere", areas_sphere_),
        ("areas_wgs84", areas_wgs84_),
        ("areas_lazy_wgs84", areas_lazy_wgs84_),
    ])
}

In [ ]:
use std::fs::File;
use parquet::{arrow::ArrowWriter, format::FileMetaData, errors::ParquetError};

fn write_to_parquet(path: String, batch: &RecordBatch) -> Result<FileMetaData, ParquetError> {
    let file = File::create(path).unwrap();
    let mut writer = ArrowWriter::try_new(file, batch.schema(), None).unwrap();
    writer.write(batch).expect("Writing batch");

    // writer must be closed to write footer
    writer.close()
}

In [ ]:
let layer = nested::get(3);
let step = 50;

let batch_3 = compute_area_stats(&layer, step, &sphere, &wgs84, &ellipsoid, &coefficients).unwrap();
write_to_parquet(format!("cell-areas-depth-{}.parquet", layer.depth()), &batch_3).unwrap();

In [ ]:
let layer = nested::get(6);
let step = 50;

let batch_6 = compute_area_stats(&layer, step, &sphere, &wgs84, &ellipsoid, &coefficients).unwrap();
write_to_parquet(format!("cell-areas-depth-{}.parquet", layer.depth()), &batch_6).unwrap();

In [ ]:
let layer = nested::get(10);
let step = 50;

let batch_10 = compute_area_stats(&layer, step, &sphere, &wgs84, &ellipsoid, &coefficients).unwrap();
write_to_parquet(format!("cell-areas-depth-{}.parquet", layer.depth()), &batch_10).unwrap();

In [ ]:
let layer = nested::get(11);
let step = 10;

let batch_11 = compute_area_stats(&layer, step, &sphere, &wgs84, &ellipsoid, &coefficients).unwrap();
write_to_parquet(format!("cell-areas-depth-{}.parquet", layer.depth()), &batch_11).unwrap();